# MQTT bridged workers - demo

## Getting Started
What this notebook does:  
- Using a client on PC
- List connected nodes
- Send messages to remote nodes

In [1]:
import os
import sys
import time
 
sys.path.append(os.path.abspath(os.path.join('..', '..', '..', 'external', 'private')))
sys.path.append(os.path.abspath(os.path.join('..', '..', 'codes', 'amd64')))
sys.path.append(os.path.abspath(os.path.join('..', '..', 'codes', 'client')))
sys.path.append(os.path.abspath(os.path.join('..', '..', 'codes', 'config')))
sys.path.append(os.path.abspath(os.path.join('..', '..', 'codes', 'micropython')))
sys.path.append(os.path.abspath(os.path.join('..', '..', 'codes', 'node')))
 
import client
from collections import OrderedDict

### Start client

In [2]:
the_client = client.Client()
the_client.start()

while not the_client.status['Is connected']:            
    time.sleep(1)
    print('Node not ready yet.')

My name is Wei-Legion

Sending 281 bytes
Message:
OrderedDict([('command', 'set connection name'), ('correlation_id', '2018-04-10 13:09:05.503151 1'), ('kwargs', {'name': 'Wei-Legion'}), ('message_id', '2018-04-10 13:09:05.503151 1'), ('message_type', 'command'), ('need_result', True), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


[Connected: ('123.240.210.68', 1883)]
[Listen to messages]
Node not ready yet.


### Prepare messages

In [3]:
# messages _____________________________________________
messages = OrderedDict()

messages['read_GPIOs'] = {'message_type': 'command',
                          'command': 'read GPIOs',
                          'kwargs': {'pins': [5, 12, 13, 14, 15, 16]},
                          'need_result': True}

messages['blink_led'] = {'message_type': 'command',
                         'command': 'blink led',
                         'kwargs': {'times': 3, 'forever': False,
                                    'on_seconds': 0.1, 'off_seconds': 0.1}}

### List connected nodes

In [4]:
the_client.node.worker.roll_call()
time.sleep(2)
remote_nodes = sorted(the_client.node.worker.contacts.keys())

print('\n[____________ Connected nodes ____________]\n')
print('\nConnected nodes:\n{}\n'.format(remote_nodes))


Sending 255 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 13:09:06.386486 10'), ('function', 'check_in'), ('kwargs', {'caller': 'Wei-Legion'}), ('message_id', '2018-04-10 13:09:06.386486 10'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Data received: 255 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 13:09:06.386486 10'), ('function', 'check_in'), ('kwargs', {'caller': 'Wei-Legion'}), ('message_id', '2018-04-10 13:09:06.386486 10'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Sending 172 bytes

Data received: 172 bytes
Message:
OrderedDict([('function', 'register_contact'), ('kwargs', {'contact_id': 'Wei-Legion', 'name': 'Wei-Legion'}), ('message_type', 'function'), ('receiver', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Data received: 196 bytes
Message:
OrderedDict([('function', 'register_contact'), ('kwargs', {'name': 'ESP32_30aea

### Blink each node
[YouTube video clip](https://youtu.be/I_s-no_0L24)

In [5]:
# for remote_node in remote_nodes:
#     the_client.request(remote_node, messages['blink_led']) 

In [6]:
the_client.request('Hub', messages['blink_led']) 


Sending 299 bytes
Message:
OrderedDict([('command', 'blink led'), ('correlation_id', '2018-04-10 13:09:08.441982 20'), ('kwargs', {'times': 3, 'forever': False, 'on_seconds': 0.1, 'off_seconds': 0.1}), ('message_id', '2018-04-10 13:09:08.441982 20'), ('message_type', 'command'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])



({'sender': 'Wei-Legion',
  'receiver': 'Hub',
  'message_type': 'command',
  'message_id': '2018-04-10 13:09:08.441982 20',
  'command': 'blink led',
  'kwargs': {'times': 3,
   'forever': False,
   'on_seconds': 0.1,
   'off_seconds': 0.1},
  'reply_to': 'Wei-Legion',
  'correlation_id': '2018-04-10 13:09:08.441982 20'},
 None)

### Read one GPIO pin

In [7]:
for remote_node in remote_nodes:
    _, result = the_client.request(remote_node, messages['read_GPIOs']) 
    print('\nGPIO status for {}: {}\n'.format(remote_node, result.get()))


Sending 300 bytes
Message:
OrderedDict([('command', 'read GPIOs'), ('correlation_id', '2018-04-10 13:09:08.488604 23'), ('kwargs', {'pins': [5, 12, 13, 14, 15, 16]}), ('message_id', '2018-04-10 13:09:08.488604 23'), ('message_type', 'command'), ('need_result', True), ('receiver', 'ESP32_30aea44cbdc8'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Data received: 299 bytes
Message:
OrderedDict([('command', 'blink led'), ('correlation_id', '2018-04-10 13:09:08.441982 20'), ('kwargs', {'times': 3, 'forever': False, 'on_seconds': 0.1, 'off_seconds': 0.1}), ('message_id', '2018-04-10 13:09:08.441982 20'), ('message_type', 'command'), ('receiver', 'Hub'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Data received: 259 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 13:09:08.488604 23'), ('message_id', '1760253'), ('message_type', 'result'), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_30aea44cbdc8'), ('result', [[5, 1], [12, 0], [13, 0], [14, 1], [15, 1]

### Send out messages and get asynchonous results

In [8]:
print('\n[______________ Sending messages ______________]\n')

results = []

# send out the messages
for message in messages.values():
    for remote_node in remote_nodes:
        if remote_node != the_client.node.worker.name:
            time.sleep(0.1)  # PyCharm needs this delay.
            formatted_message, asynch_result = the_client.request(remote_node, message)
            results.append((formatted_message, asynch_result))


[______________ Sending messages ______________]


Sending 302 bytes
Message:
OrderedDict([('command', 'read GPIOs'), ('correlation_id', '2018-04-10 13:09:10.099206 184'), ('kwargs', {'pins': [5, 12, 13, 14, 15, 16]}), ('message_id', '2018-04-10 13:09:10.099206 184'), ('message_type', 'command'), ('need_result', True), ('receiver', 'ESP32_30aea44cbdc8'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])


Sending 316 bytes
Message:
OrderedDict([('command', 'blink led'), ('correlation_id', '2018-04-10 13:09:10.232916 187'), ('kwargs', {'times': 3, 'forever': False, 'on_seconds': 0.1, 'off_seconds': 0.1}), ('message_id', '2018-04-10 13:09:10.232916 187'), ('message_type', 'command'), ('receiver', 'ESP32_30aea44cbdc8'), ('reply_to', 'Wei-Legion'), ('sender', 'Wei-Legion')])



### Actually get the results

In [9]:
# collect and print results        
print('\n[_________ Wait few seconds for reply _________]\n')
for (message, result) in results:
    try:
        if message.get('need_result'):
            print('\n[Result for request]:\n___Request___:\n{0}\n___Result____:\n{1}\n'.format(message,
                                                                                               result.get() if result else None))
    except Exception as e:
        print('\n[{}]\nMessage:\n{}'.format(e, message))
        
# Wait a while
time.sleep(3)        


[_________ Wait few seconds for reply _________]


Data received: 260 bytes
Message:
OrderedDict([('correlation_id', '2018-04-10 13:09:10.099206 184'), ('message_id', '1761280'), ('message_type', 'result'), ('receiver', 'Wei-Legion'), ('reply_to', 'ESP32_30aea44cbdc8'), ('result', [[5, 1], [12, 0], [13, 0], [14, 1], [15, 1], [16, 0]]), ('sender', 'ESP32_30aea44cbdc8')])


[Result for request]:
___Request___:
{'sender': 'Wei-Legion', 'receiver': 'ESP32_30aea44cbdc8', 'message_type': 'command', 'message_id': '2018-04-10 13:09:10.099206 184', 'command': 'read GPIOs', 'kwargs': {'pins': [5, 12, 13, 14, 15, 16]}, 'need_result': True, 'reply_to': 'Wei-Legion', 'correlation_id': '2018-04-10 13:09:10.099206 184'}
___Result____:
[[5, 1], [12, 0], [13, 0], [14, 1], [15, 1], [16, 0]]



### Stop the demo

In [10]:
# Stopping
the_client.stop()
the_client = None
print('\n[________________ Demo stopped ________________]\n')

[Closed: ('123.240.210.68', 1883)]

[________________ Demo stopped ________________]

